<a href="https://colab.research.google.com/github/AgungMaruti/project/blob/main/projectasli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
app_code = """
# Import semuanya dulu
import streamlit as st
from bot import build_agent

# Judul
st.title("Bo Bots")

# Session state
if "agent" not in st.session_state:
  st.session_state.agent = build_agent()

if "messages" not in st.session_state:
  st.session_state.messages = []

agent = st.session_state.agent

# Tombol reset
reset_chat_button = st.button("reset chat")
if reset_chat_button:
  st.session_state.messages = []
  st.session_state.agent = build_agent()

user_input = st.chat_input()

for m in st.session_state.messages:
  with st.chat_message(m["role"]):
    st.markdown(m["content"], unsafe_allow_html=True)

if user_input is not None:
  st.session_state.messages.append({
    "role": "human",
    "content": user_input,
  })

  with st.chat_message("user"):
    st.markdown(user_input)

  with st.spinner("Thinking.."):
    ai_output = ""
    for step in agent.stream({"input": user_input}):
      if "actions" in step.keys():
        for action in step["actions"]:
          with st.chat_message("assistant"):
            tool_name = action.tool
            tool_input = action.tool_input
            tool_message = f\"\"\"<div style='border-left:5px solid #4CAF50;padding:6px;background:#f9f9f9;border-radius:4px;font-size:14px;'>🛠️ <b>{tool_name}</b> <code>{tool_input}</code></div>\"\"\"
            st.session_state.messages.append({"role": "🛠️", "content": tool_message})
            st.markdown(tool_message, unsafe_allow_html=True)

      if "output" in step.keys():
        ai_output = step["output"]

  with st.chat_message("assistant"):
    st.session_state.messages.append({
      "role": "assistant",
      "content": ai_output
    })
    st.markdown(ai_output, unsafe_allow_html=True)
"""

with open("app.py", "w") as f:
    f.write(app_code)

print("app.py created!")


app.py created!


In [ ]:
bot_code = """
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Replicate
from langchain_core.tools import tool
import requests

def parse_input(input_str):
    parts = input_str.split(";")
    return dict(part.split("=") for part in parts)

@tool
def multiply(input: str) -> str:
    try:
        input_dict = parse_input(input)
        a = float(input_dict['a'])
        b = float(input_dict['b'])
        return str(a * b)
    except Exception as e:
        return f"Error: {e}"

@tool
def cat_fact(input):
    try:
        response = requests.get("https://catfact.ninja/fact?max_length=200")
        return str(response.json()['fact'])
    except Exception as e:
        return f"Error: {e}"

@tool
def get_weather(input: str) -> str:
    try:
        input_dict = parse_input(input)
        lat = float(input_dict['lat'])
        lon = float(input_dict['lon'])
        response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true")
        return str(response.json())
    except Exception as e:
        return f"Error: {e}"

def build_agent():
    llm = Replicate(model="anthropic/claude-3.5-haiku")

    system_message = \"\"\"Kamu adalah orang aneh yang suka ngaco dan bercanda tidak nyambung, tetapi jago matematika dan tahu banyak fakta kucing serta cuaca dunia.\"\"\"

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )

    tools = [multiply, cat_fact, get_weather]

    agent_executor = initialize_agent(
        llm=llm,
        tools=tools,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        agent_kwargs={"system_message": system_message},
        verbose=True,
        max_iterations=10,
        handle_parsing_errors=True
    )
    return agent_executor
"""

with open("bot.py", "w") as f:
    f.write(bot_code)

print("bot.py created!")


bot.py created!


In [ ]:
req = """
streamlit==1.52.0
pyngrok==7.5.0
python-dotenv==1.2.1
langchain==0.2.16
langchain-core==0.2.41
langchain-community==0.2.11
langchain-text-splitters==0.2.4
langgraph==0.2.3
langchain-ollama==0.1.1
semantic-router==0.0.61
pyppeteer==2.0.0
nest-asyncio==1.6.0
praw==7.7.1
cohere==5.5.0
replicate==1.0.7
"""

with open("requirements.txt", "w") as f:
    f.write(req)

print("requirements.txt created!")


requirements.txt created!


In [ ]:
!pip install -r requirements.txt


In [ ]:
import os
os.environ["REPLICATE_API_TOKEN"] = "r8_bk428rBRDciVWHgr6Q9o4f59f4alJZ11IPvuZ"


In [ ]:
!streamlit run app.py & sleep 5





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.48.110.236:8501

  Stopping...
